In [5]:
import zipfile
import os
import pandas as pd
import json
from bs4 import BeautifulSoup
import re

In [6]:
fileZip_path_text = '/home/project/Documents/mgmt/extracted.zip'
fileZip_path_text_raw = '/home/project/Documents/mgmt/raw.zip'
fileZip_path_text_raw1 = '/home/project/Documents/mgmt/raw1.zip'

extract_dir = '/home/project/Documents/mgmt/extract_dir'

with zipfile.ZipFile(fileZip_path_text, "r") as extract_ref:
    extract_ref.extractall(extract_dir)

with zipfile.ZipFile(fileZip_path_text_raw, "r") as extract_ref:
    extract_ref.extractall(extract_dir)

with zipfile.ZipFile(fileZip_path_text_raw1, "r") as extract_ref:
    extract_ref.extractall(extract_dir)

files = os.listdir(extract_dir)
print(files[:10])

#print("Extracted 10K files folder", os.listdir(extract_dir)[:5])


['raw1', 'extracted', 'raw']


In [7]:
#Clean Text dataset(10-K reports)

records = []

for root, dirs, files in os.walk(extract_dir):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        text = ""           

In [8]:
#Clean Text dataset(10-K reports)

records = []

for root, dirs, files in os.walk(extract_dir):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        text = "" 

    #json files
    if file_name.lower().endswith(".json"):
        try:
            with open(file_path, "r", encoding="utf-8" , errors="ignore") as f:
                data = json.load(f)
                for key in ["company"]:
                    if key in data:
                        text = data[key]
                        break                     
        except Exception as e:
            print(f"Error reading JSON {file_name}: {e}")


    elif file_name.lower().endswith(".htm"):
        try:
            with open(file_path, "r", encoding="utf-8" , errors="ignore") as f:
                html = f.read()
                soup = BeautifulSoup(html, "html.parser")
                for script in soup(["script", "style"]):
                    script.decompose()
                text = soup.get_text()

        except Exception as e:
            print(f"Error reading HTML {file_name}: {e}")

    #Clean whitespaces
    text = re.sub(r"\s+", " ", text).strip()

    if text:
        records.append({"file_name": file_name, "clean_text" : text})

df_texts = pd.DataFrame(records)
print(df_texts.head())
print("Total files processed:", len(df_texts))

                                   file_name  \
0  1404655_10K_2020_0001564590-21-006083.htm   
1  1528396_10K_2021_0001528396-21-000095.htm   
2  837852_10K_2020_0001104659-21-044740.json   
3  1404655_10K_2020_0001564590-21-006083.htm   

                                          clean_text  
0  hubs-10k_20201231.htm false FY 0001404655 --12...  
1  gwre-2021073100015283962021FYfalseP3YP3Mus-gaa...  
2                                   IDEANOMICS, INC.  
3  hubs-10k_20201231.htm false FY 0001404655 --12...  
Total files processed: 4


In [9]:
df_texts.to_csv("unified_texset.csv", index=False)
print("Saved as 'unified_cleaned_text.csv'")

Saved as 'unified_cleaned_text.csv'
